In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
 
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
import os

model = os.environ["LLM_MODEL_NAME"]
api_key = os.environ["LLM_API_KEY"]
base_url = os.environ["OLLAMA_ENDPOINT"] 


model_client = OpenAIChatCompletionClient(
    model=model,
    base_url=base_url,
    api_key=api_key,
    parallel_tool_calls= False,
    model_info={
        "family": "llama",
        "vision": False,
        "json_output": True,
        "function_calling": True,
    },
)

# Create a tool for incrementing a number.
def increment_number(number: int) -> int:
    """Increment a number by 1."""
    return number + 1



web_surfer_agent = MultimodalWebSurfer(
    name="WebScraper",
    model_client=model_client,
    downloads_folder="./downloads",  # Folder where downloaded files can be saved
    description="Agent who can scrape web pages and search for information on the web.",
    debug_dir="./debug",             # Directory for saving debug output (optional)
    headless=True,                   # Run the browser in headless mode
    start_page="https://www.google.com",  # The page where the browser should start
    animate_actions=False,           # Do not animate actions by default
    to_save_screenshots=True,        # Save screenshots during navigation (if needed)
    use_ocr=False,                   # Do not use OCR by default
    browser_channel=None,            # Use the default browser channel
    browser_data_dir=None,           # Use the default data directory for the browser
    to_resize_viewport=True,         # Enable viewport resizing if required
    playwright=None,                 # Let the agent initialize its own Playwright instance
    context=None                     # Use the default browser context
)


text_termination = TextMentionTermination("APPROVE")
 
assistant_agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You are an advanced AI agent who can handel customer requests critically and communicate with other agents. Respond with 'APPROVE' to when your feedbacks are addressed. Please respond in English for all queries.",
    model_client_stream= True
)

primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant. Don't include additional words or phrase. Simply provide the answer that user wants.",
)


critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Only Respond with 'APPROVE' to when your feedbacks are addressed. Otherwise, don't say 'APPROVE' in your responses. ",
)



In [11]:
agents = [primary_agent, critic_agent]
web_handler_agents = [primary_agent,assistant_agent, web_surfer_agent ]
team = RoundRobinGroupChat(agents, termination_condition=text_termination)
 
await team.reset()  # Reset the team for a new task.
# async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
#     if isinstance(message, TaskResult):
#         print("Stop Reason:", message.stop_reason)
#     else:
#         print(message)

await Console(stream=team.run_stream(task="Write a short poem about the fall season."),output_stats= True )

---------- user ----------
Write a short poem about the fall season.


---------- primary ----------
Golden leaves rustle down,
Summer's warmth has turned to frown,
Crisp air bites with gentle chill,
Fall's arrival, Nature's will.

Scarlet maples stand so tall,
Orange hues against the wall,
Crimson red and amber bright,
A fleeting colorful sight.
[Prompt tokens: 47, Completion tokens: 60]
---------- critic ----------
Your poem effectively captures the beauty of fall through vibrant imagery and concise language. 

You might consider varying your stanza structure to create more dynamic rhythm. Currently, both stanzas follow a similar pattern of four lines, which can make them feel a bit repetitive.

Here's an example of how you could revise your second stanza using an ABA rhyme scheme and different line lengths:

Scarlet maples shine so bright
In rustling crowds of golden leaves
Their autumn hue a wondrous sight
A season's farewell, Nature's eaves
[Prompt tokens: 103, Completion tokens: 109]
---------- primary ----------
Golden leaves rustle down,
Summer's 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=60), metadata={}, content="Golden leaves rustle down,\nSummer's warmth has turned to frown,\nCrisp air bites with gentle chill,\nFall's arrival, Nature's will.\n\nScarlet maples stand so tall,\nOrange hues against the wall,\nCrimson red and amber bright,\nA fleeting colorful sight.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=103, completion_tokens=109), metadata={}, content="Your poem effectively captures the beauty of fall through vibrant imagery and concise language. \n\nYou might consider varying your stanza structure to create more dynamic rhythm. Currently, both stanzas follow a similar pattern of four lines, which can make them feel a bit repetitive.\n\nHere's an example of how you could rev